In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
palettes=pd.read_csv('../input/train_2.csv')
palettes.head()

In [ ]:
# data processsing
y=palettes['style']
X=palettes.drop('style',axis=1)

# label encoding
le=preprocessing.LabelEncoder()
le.fit(y)
y=le.transform(y)

In [ ]:
y.value_counts()

In [ ]:
train_X,test_X,train_Y,test_Y=train_test_split(X,y,test_size=0.25,random_state=1)

# create dataset for lightgbm
lgb_train=lgb.Dataset(train_X,train_Y)
lgb_eval=lgb.Dataset(test_X,test_Y,reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [ ]:
print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

In [ ]:
print('Saving model...')
# save model to file
gbm.save_model('model.txt')

In [ ]:

print('Starting predicting...')
# predict
y_pred = gbm.predict(test_X, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(test_Y, y_pred) ** 0.5)

In [ ]:
train_X.head()

In [ ]:
print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)
print('Starting predicting...')
# predict
y_pred = gbm.predict(test_X, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(test_Y, y_pred) ** 0.5)